In [71]:
import re
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import glob
import os

## Read the 4 csv files:
- Laptop data from 2 websites: thinkpro and laptop88
- GPU infos include its score
- CPU infos include avg_bench, gamming, desktop, workstation score

In [401]:
df_cpu = pd.read_csv("../data/cpu_info.csv", index_col=0)

In [467]:
df = pd.read_csv("../data/laptop_data.csv")
df["website"] = "thinkpro"

In [402]:
df_gpu = pd.read_csv('../data/gpu_map.txt', sep=":", header=None)
df_gpu.columns = ["gpu", "gpu_score"]

In [406]:
df_gpu.head()

,gpu,gpu_score
0,0,0.00
1,NVIDIA GeForce RTX 3050 Ti,58.50
2,Intel Iris Xe Graphics,8.34
3,NVIDIA® GeForce RTX™ 3050,53.70
4,AMD Radeon Graphics,3.59


In [403]:
df_cpu.head()

,cpu_name,number_of_core,number_of_thread,avg_bench,gamming,desktop,workstation
0,0,0,0,0.0,0.0,0.0,0.0
1,Intel Core i7-12700H,14,20,98.1,98.0,100.0,104.0
2,Intel Core i7-1260P,12,16,73.8,74.0,80.0,70.0
3,Intel Core i5-1235U,10,12,63.6,64.0,72.0,59.0
4,AMD AMD Ryzen™ 7 6800H,8,16,78.1,78.0,81.0,77.0


In [404]:
df_cpu["number_of_core"] = df_cpu["number_of_core"].str.replace(" Cores", "")
df_cpu["number_of_thread"] = df_cpu["number_of_thread"].str.replace(" Threads", "")

In [405]:
df_cpu["number_of_core"].value_counts()

4     59
8     47
6     36
10    16
14    13
2     13
12    12
0      9
16     5
Name: number_of_core, dtype: int64

In [407]:
pd.set_option('display.max_colwidth', None)

In [408]:
df_laptop88 = pd.read_excel("../data/laptop88.xlsx")
df_laptop88["website"] = "laptop88"

In [409]:
df_laptop88.head(5)

,NAME,CPU,RAM,DISK,GPU,SIZE,BATTERY,WEIGHT,PRICE,website
0,[Mới 100% Full Box] Laptop Lenovo Ideapad Gaming 3 15ACH6 82K201XCUS - AMD Ryzen 5 - 5600H | RTX 3050 Ti | 15.6 Inch Full HD 120Hz,R5 - 5600H,8GB DDR4,SSD 256GB NVMe,Nvidia RTX 3050 Ti,"15.6"" FHD 120Hz",grated 45wh,around 2.25,16.790.000 đ,laptop88
1,[Mới 100% Full Box] Laptop Gaming GIGABYTE G5 MD-51US113SO - Intel Core i5-11400H | RTX 3050Ti | 144Hz,i5 - 11400H,8GB DDR4,SSD 512GB NMVe,Nvidia RTX 3050Ti,"15.6"" Full HD 144Hz",ion 48.96wh,2.2,17.690.000 đ,laptop88
2,[Mới 99% Full-Box] Laptop Acer Nitro 5 AN515-58-5046 - Intel Core i5-12500H | RAM 16GB | Nvidia RTX 3050Ti | 15.6 Inch Full HD 144Hz,i5 - 12500H,16GB DDR4,SSD 512GB,Nvidia RTX 3050Ti,"15.6"" Full HD 144Hz",NaN,2.3,20.490.000 đ,laptop88
3,[Mới 99% Full-Box] Laptop Acer Nitro 5 Eagle AN515-57-536Q / 56FC - Intel Core i5 11400H GTX 1650,i5 - 11400H,8GB DDR4,SSD 256GB,Nvidia GTX 1650,"15.6"" Full HD 144Hz",NaN,2.3,15.590.000 đ,laptop88
4,[Mới 100% Full Box] Laptop Lenovo IdeaPad 5 Pro 16ACH6 82L500F5US - AMD Ryzen 5-5600H | 16 Inch WQXGA 100% sRGB,AMD R5-5600H,8GB DDR4,SSD 512GB,AMD Radeon Vega,"16"" WQXGA 100% sRGB",grated 75wh,around 1.9,15.690.000 đ,laptop88


In [410]:
df_laptop88.columns = df_laptop88.columns.str.lower()
df_laptop88.rename(columns = {"size": "monitor", "disk":"memory"}, inplace = True)

Drop the duplicated rows

In [411]:
df_laptop88 = df_laptop88.drop_duplicates(ignore_index = True)

In [412]:
df_laptop88.shape

(559, 10)

In [413]:
df_laptop88["price"]

0      16.790.000 đ
1      17.690.000 đ
2      20.490.000 đ
3      15.590.000 đ
4      15.690.000 đ
           ...     
554         Liên Hệ
555         Liên Hệ
556         Liên Hệ
557         Liên Hệ
558         Liên Hệ
Name: price, Length: 559, dtype: object

In [414]:
def process_price(x):
    try:
        ret = re.sub(r"\D", "", x)
        if ret == "":
            return np.nan
        else:
            return ret
    except:
        return x

Remove all instances with missing price value or have more than 4 missing attributes

In [415]:
df_laptop88["price"] = df_laptop88["price"].apply(process_price)
df_laptop88 = df_laptop88.dropna(subset = ['price'], how = "any")
df_laptop88 = df_laptop88.dropna(thresh = 5)
df_laptop88 = df_laptop88.reset_index(drop=True)
df_laptop88

,name,cpu,ram,memory,gpu,monitor,battery,weight,price,website
0,[Mới 100% Full Box] Laptop Lenovo Ideapad Gaming 3 15ACH6 82K201XCUS - AMD Ryzen 5 - 5600H | RTX 3050 Ti | 15.6 Inch Full HD 120Hz,R5 - 5600H,8GB DDR4,SSD 256GB NVMe,Nvidia RTX 3050 Ti,"15.6"" FHD 120Hz",grated 45wh,around 2.25,16790000,laptop88
1,[Mới 100% Full Box] Laptop Gaming GIGABYTE G5 MD-51US113SO - Intel Core i5-11400H | RTX 3050Ti | 144Hz,i5 - 11400H,8GB DDR4,SSD 512GB NMVe,Nvidia RTX 3050Ti,"15.6"" Full HD 144Hz",ion 48.96wh,2.2,17690000,laptop88
2,[Mới 99% Full-Box] Laptop Acer Nitro 5 AN515-58-5046 - Intel Core i5-12500H | RAM 16GB | Nvidia RTX 3050Ti | 15.6 Inch Full HD 144Hz,i5 - 12500H,16GB DDR4,SSD 512GB,Nvidia RTX 3050Ti,"15.6"" Full HD 144Hz",NaN,2.3,20490000,laptop88
3,[Mới 99% Full-Box] Laptop Acer Nitro 5 Eagle AN515-57-536Q / 56FC - Intel Core i5 11400H GTX 1650,i5 - 11400H,8GB DDR4,SSD 256GB,Nvidia GTX 1650,"15.6"" Full HD 144Hz",NaN,2.3,15590000,laptop88
4,[Mới 100% Full Box] Laptop Lenovo IdeaPad 5 Pro 16ACH6 82L500F5US - AMD Ryzen 5-5600H | 16 Inch WQXGA 100% sRGB,AMD R5-5600H,8GB DDR4,SSD 512GB,AMD Radeon Vega,"16"" WQXGA 100% sRGB",grated 75wh,around 1.9,15690000,laptop88
...,...,...,...,...,...,...,...,...,...,...
304,[Mới 100% Full Box] Laptop Gigabyte U4 UD 70S1823SO - Intel Core i7,Core i7-1195G7,16GB DDR4,SSD 512GB NVMe,Intel Iris Xe Graphics,"14"" Full HD",eyboard (white),NaN,25990000,laptop88
305,Laptop Cũ Dell Dell XPS 13 9380 - Intel Core i7,i7 - 8560U,16GB Onmain,SSD 256GB,Intel UHD Graphics,"13.3"" Full HD",NaN,1.2,19990000,laptop88
306,[Mới 100% Full Box] Laptop GIGABYTE AORUS 15P YD-73S1224GH - Intel Core i7,Core i7-11800H,16GB DDR4,SSD 1TB NVMe,Nvidia RTX 3080,"15.6"" Full HD 240Hz",ell - 99 wh,2.,58990000,laptop88
307,[Mới 100% Full Box] Laptop Alienware m15 R6 P109F001BBL - Intel Core i7,Core i7 - 11800H,32GB DDR4,SSD 1TB NVMe,RTX 3060 6GDDR6,"15.6"" Full HD 165Hz",cell - 86wh,2.69,58990000,laptop88


In [416]:
df_laptop88["name"].str.lower().str.contains("cũ").value_counts()

False    302
True       7
Name: name, dtype: int64

Drop all the rows containing old (secondhand) laptops 

In [417]:
df_laptop88 = df_laptop88.drop(axis = 0, index = df_laptop88[df_laptop88["name"].str.lower().str.contains("cũ")].index)

In [418]:
monitor_new = df_laptop88["monitor"].str.split(r" |\"").to_frame()
monitor_new

,monitor
0,"[15.6, , FHD, 120Hz]"
1,"[15.6, , Full, HD, 144Hz]"
2,"[15.6, , Full, HD, 144Hz]"
3,"[15.6, , Full, HD, 144Hz]"
4,"[16, WQXGA 100%, sRGB]"
...,...
303,"[15.6, , Full, HD, 165Hz]"
304,"[14, , Full, HD]"
306,"[15.6, , Full, HD, 240Hz]"
307,"[15.6, , Full, HD, 165Hz]"


#### Handling the monitor attribute to get the size for the monitors

In [419]:
def size(x):
    try:
        return x[0].replace("\"", "")
    except:
        return x

In [420]:
df_laptop88["monitor_size"] = monitor_new["monitor"].apply(size)

In [421]:
df_laptop88["monitor_size"]

0      15.6
1      15.6
2      15.6
3      15.6
4        16
       ... 
303    15.6
304      14
306    15.6
307    15.6
308      14
Name: monitor_size, Length: 302, dtype: object

In [422]:
df_laptop88["ram"].value_counts()

8GB DDR4            119
16GB DDR4            48
16GB DDR5            26
8GB DDR5             22
4GB DDR4             19
16GB LPDDR5          13
32GB DDR5            11
8GB LPDDR5            8
16GB LPDDR5           6
8GB LPDDR4            4
16GB                  3
16GB DDR4             3
8GB                   3
32GB DDR4             2
32GB LPDDR5           2
16GB DDR5             2
8GB DDR5              2
16GB LPDDR4           1
24GB DDR4             1
32GB DDR4             1
8GB DDR4              1
8GB LPDDR4            1
8GB DDR4 3200MHz      1
16GB  DDR4            1
8GB DDR4 3200Mhz      1
12GB DDR4             1
Name: ram, dtype: int64

In [423]:
def bs_replace(x):
    return x.replace(u'\xa0', u' ')

In [424]:
df_laptop88["ram"] = df_laptop88["ram"].apply(bs_replace)

In [425]:
ram_new = df_laptop88["ram"].str.split(r", | |  ").to_frame()

In [426]:
ram_new

,ram
0,"[8GB, DDR4]"
1,"[8GB, DDR4]"
2,"[16GB, DDR4]"
3,"[8GB, DDR4]"
4,"[8GB, DDR4]"
...,...
303,"[16GB, DDR4]"
304,"[16GB, DDR4]"
306,"[16GB, DDR4]"
307,"[32GB, DDR4]"


In [427]:
def capacity(x):
    try: 
        for i in x:
            if "GB" in i:
                return i.replace("GB", "")
        return np.nan
    except:
        return x

def ram_type(x):
    try: 
        for i in x:
            if i == "DDR4":
                return i
            if i == "LPDDR5":
                return i
            if "DDR5" in i:
                return i
            if i == "LPDDR4x":
                return i
            if i == "LPDDR3L":
                return i
            if i == "LPDDR3":
                return i
            if i == "DDR3L":
                return i
        return np.nan
    except:
        return x

#### Handling the ram attribute to get the capacity and the type of each ram

In [428]:
df_laptop88["ram_capacity"] = ram_new["ram"].apply(capacity)
df_laptop88["ram_type"] = ram_new["ram"].apply(ram_type)

In [429]:
df_laptop88["ram_type"].value_counts(dropna=False)

DDR4      198
DDR5       63
LPDDR5     29
NaN        12
Name: ram_type, dtype: int64

#### Handling the memory attribute to get the size of the HDD and SSD disk

In [430]:
df_laptop88["memory"].value_counts()

SSD 512GB NVMe         71
SSD 512GB NVMe         42
SSD 256GB NVMe         32
SSD 256GB              26
SSD 512GB              25
SSD 256GB NVMe         24
SSD 512GB NMVe         20
SSD 1TB NVMe           11
SSD 512GB NVMe          9
SSD 1TB NVMe            8
SSD 1TB                 7
SSD 256GB NMVe          5
SSD 2TB NVMe            3
SSD 1TB NMVe            2
SSD 128GB               1
SSD 2TB NVMe            1
1TB SATA HDD            1
SSD1TB PCIE             1
SSD 128GB NVMe          1
SSD 512GB               1
SSD 512GB M.2 NMVe      1
256GB NVMe              1
SSD 512GB + HDD 1TB     1
SSD 128GB  NVMe         1
1TB M.2 NVMe™ SSD       1
SSD512GB NVMe           1
SSD 1TB NVMe            1
1TB SSD NVMe            1
SSD 1TB PCIe            1
SSD 1TB PCIe            1
SSD 1TB M.2 NVMe        1
Name: memory, dtype: int64

In [431]:
df_laptop88["memory"] = df_laptop88["memory"].str.replace("HDD 0GB", "").str.strip(" , ")
df_laptop88["memory"] = df_laptop88["memory"].str.replace("1TB", "1024GB")
df_laptop88["memory"] = df_laptop88["memory"].str.replace("2TB", "2048GB")
df_laptop88["memory"] = df_laptop88["memory"].str.replace("1.9765625TB", "2048GB")
df_laptop88["memory"] = df_laptop88["memory"].str.replace("1.001953125TB", "1024GB")
df_laptop88["memory"] = df_laptop88["memory"].str.replace("4TB", "4096GB")

df_laptop88.loc[df_laptop88["memory"] == "","memory"] = np.nan

<ipython-input-431-b95e6630919d>:4: FutureWarning: The default value of regex will change from True to False in a future version.
  df_new["memory"] = df_new["memory"].str.replace("1.9765625TB", "2048GB")
<ipython-input-431-b95e6630919d>:5: FutureWarning: The default value of regex will change from True to False in a future version.
  df_new["memory"] = df_new["memory"].str.replace("1.001953125TB", "1024GB")


In [432]:
memory_new= df_laptop88["memory"].str.split(r",|/+").to_frame()
memory_new

,memory
0,[SSD 256GB NVMe]
1,[SSD 512GB NMVe]
2,[SSD 512GB]
3,[SSD 256GB]
4,[SSD 512GB]
...,...
303,[SSD 1024GB NVMe]
304,[SSD 512GB NVMe]
306,[SSD 1024GB NVMe]
307,[SSD 1024GB NVMe]


In [433]:
def hdd_capacity(x):
    try:
        for i in x:
            if "HDD" in i:
                return re.sub(r"\D", "", i)
        return 0
    except:
        return x

def ssd_capacity(x):
    try:
        for i in x:
            if "SSD" in i:
                return re.sub(r"\D", "", i)
        return 0
    except:
        return x

In [434]:
df_laptop88["HDD_capacity"] = memory_new["memory"].apply(hdd_capacity)
df_laptop88["SSD_capacity"] = memory_new["memory"].apply(ssd_capacity)

In [435]:
df_laptop88["SSD_capacity"].value_counts()

512        169
256         87
1024        33
2048         4
128          3
0            2
10242        2
5122         1
5121024      1
Name: SSD_capacity, dtype: int64

#### Weight attribute

In [436]:
def process_weight(x):
    try:
        for i in x:
            return re.sub(r"\D", "", i)
        return 0
    except:
        return x

In [437]:
df_laptop88["weight"] = df_laptop88["weight"].apply(process_weight)

In [438]:
df_laptop88["weight"].value_counts()

1    112
2     68
      25
3      1
Name: weight, dtype: int64

In [439]:
df_laptop88["weight"] = df_laptop88["weight"].replace("", np.nan)

#### Battery attribute

In [440]:
battery_new = df_laptop88["battery"].to_frame()

In [441]:
def process_battery(x):
    try:
        bat_split = x.split(" ")
        for i in range(len(bat_split )):
            if "wh" in bat_split [i] and " wh" != bat_split [i]:
                ret = re.sub(r"whrs|whr|wh", "", bat_split [i])
                return re.sub("[^0-9.]", "", ret)
            elif " wh" == bat_split [i]:
                ret =  bat_split [i-1] 
                return re.sub("/[^0-9\-\.]+/g", "", ret)
        return np.nan
    except:
        return x
    # return bat_split
    
def post_bat(x):
    try:
        if x == "" or x == "0":
            return np.nan
        else:
            return x
    except: 
        return x


In [442]:
df_laptop88["battery"] = battery_new["battery"].apply(process_battery)

In [443]:
df_laptop88["battery"] = df_laptop88["battery"].apply(post_bat)

#### Merge the cpu of the laptops with the corresponding score (including avg_bench, gamming, desktop, workstation)

In [444]:
df_cpu = df_cpu.drop_duplicates(ignore_index = True, subset = ["cpu_name"])
with pd.option_context('display.max_rows', 10,):
    print(df_cpu["cpu_name"].str.replace("-", " "))

0                                    0
1                 Intel Core i7 12700H
2                  Intel Core i7 1260P
3                  Intel Core i5 1235U
4               AMD AMD Ryzen™ 7 6800H
                    ...               
186              Intel Core i9 12900HX
187               Intel Core i9 12900H
188              Intel Core i9 11980HK
189               Intel Core i9 11900H
190    Intel Core i9 12900HX Processor
Name: cpu_name, Length: 191, dtype: object


In [445]:
df_laptop88["cpu"] = df_laptop88["cpu"].apply(bs_replace)

In [446]:
def get_cpu_brand(x):
    try:
        if "AMD" in x:
            return "AMD"
        elif "Intel" in x or bool(re.search(r"i.|i .",x)):
            return "Intel"
        elif "Apple" in x:
            return "Apple"
    except:
        return np.nan

def get_cpu_level(x):
    try:
        match = re.search('i\d{1}|i \d{1}|R\d{1}| \d ', x)
        return match.group(0)
    except:
        return np.nan

def get_cpu_ver(x):
    try:
        match = re.search('\d{5}\w{2}|\d{5}\w{1}|\d{4}\w{2}|\d{4}\w{1}|\w{1}\d{4}|\d{4}|M1|M2|M1 Pro ', x)
        return match.group(0)
    except:
        return np.nan

In [447]:
df_cpu["brand"] = df_cpu["cpu_name"].apply(get_cpu_brand)
df_cpu["level"] = df_cpu["cpu_name"].apply(get_cpu_level)
df_cpu["ver"] = df_cpu["cpu_name"].apply(get_cpu_ver)
df_cpu = df_cpu.drop_duplicates(subset = ["brand", "level", "ver"], ignore_index = True)
df_cpu.shape

(136, 10)

In [448]:
df_cpu

,cpu_name,number_of_core,number_of_thread,avg_bench,gamming,desktop,workstation,brand,level,ver
0,0,0,0,0.0,0.0,0.0,0.0,None,NaN,NaN
1,Intel Core i7-12700H,14,20,98.1,98.0,100.0,104.0,Intel,i7,12700H
2,Intel Core i7-1260P,12,16,73.8,74.0,80.0,70.0,Intel,i7,1260P
3,Intel Core i5-1235U,10,12,63.6,64.0,72.0,59.0,Intel,i5,1235U
4,AMD AMD Ryzen™ 7 6800H,8,16,78.1,78.0,81.0,77.0,AMD,7,6800H
...,...,...,...,...,...,...,...,...,...,...
131,Intel Core i9-12900HK,14,20,101.0,101.0,104.0,106.0,Intel,i9,12900HK
132,Intel Core i9-12900HX,16,24,103.0,103.0,106.0,116.0,Intel,i9,12900HX
133,Intel Core i7 10700K,8,16,96.5,97.0,97.0,96.0,Intel,i7,10700K
134,Intel Core i9 11980HK,8,16,93.0,93.0,95.0,94.0,Intel,i9,11980HK


In [449]:
df_laptop88["brand"] = df_laptop88["cpu"].apply(get_cpu_brand)
df_laptop88["level"] = df_laptop88["cpu"].apply(get_cpu_level)
df_laptop88["ver"] = df_laptop88["cpu"].apply(get_cpu_ver)

In [450]:
df_laptop88 = df_laptop88.merge(df_cpu, how = "left", on = ["level", "ver"])

In [451]:
df_laptop88

,name,cpu,ram,memory,gpu,monitor,battery,weight,price,website,...,level,ver,cpu_name,number_of_core,number_of_thread,avg_bench,gamming,desktop,workstation,brand_y
0,[Mới 100% Full Box] Laptop Lenovo Ideapad Gaming 3 15ACH6 82K201XCUS - AMD Ryzen 5 - 5600H | RTX 3050 Ti | 15.6 Inch Full HD 120Hz,R5 - 5600H,8GB DDR4,SSD 256GB NVMe,Nvidia RTX 3050 Ti,"15.6"" FHD 120Hz",45,NaN,16790000,laptop88,...,R5,5600H,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,[Mới 100% Full Box] Laptop Gaming GIGABYTE G5 MD-51US113SO - Intel Core i5-11400H | RTX 3050Ti | 144Hz,i5 - 11400H,8GB DDR4,SSD 512GB NMVe,Nvidia RTX 3050Ti,"15.6"" Full HD 144Hz",48.96,2,17690000,laptop88,...,i5,11400H,Intel Core i5 11400H,6,12,74.5,74.0,78.0,69.0,Intel
2,[Mới 99% Full-Box] Laptop Acer Nitro 5 AN515-58-5046 - Intel Core i5-12500H | RAM 16GB | Nvidia RTX 3050Ti | 15.6 Inch Full HD 144Hz,i5 - 12500H,16GB DDR4,SSD 512GB,Nvidia RTX 3050Ti,"15.6"" Full HD 144Hz",NaN,2,20490000,laptop88,...,i5,12500H,Intel Core i5-12500H,12,16,94.8,95.0,97.0,94.0,Intel
3,[Mới 99% Full-Box] Laptop Acer Nitro 5 Eagle AN515-57-536Q / 56FC - Intel Core i5 11400H GTX 1650,i5 - 11400H,8GB DDR4,SSD 256GB,Nvidia GTX 1650,"15.6"" Full HD 144Hz",NaN,2,15590000,laptop88,...,i5,11400H,Intel Core i5 11400H,6,12,74.5,74.0,78.0,69.0,Intel
4,[Mới 100% Full Box] Laptop Lenovo IdeaPad 5 Pro 16ACH6 82L500F5US - AMD Ryzen 5-5600H | 16 Inch WQXGA 100% sRGB,AMD R5-5600H,8GB DDR4,SSD 512GB,AMD Radeon Vega,"16"" WQXGA 100% sRGB",75,NaN,15690000,laptop88,...,R5,5600H,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
312,[Mới 100% Full Box] Laptop Gaming Dell Alienware M15 R5 Ryzen Edition - R9 5900HX RTX 3070,AMD R9-5900HX,16GB DDR4,SSD 1024GB NVMe,Nvidia RTX 3070,"15.6"" Full HD 165Hz",NaN,2,54990000,laptop88,...,R9,5900HX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
313,[Mới 100% Full Box] Laptop Gigabyte U4 UD 70S1823SO - Intel Core i7,Core i7-1195G7,16GB DDR4,SSD 512GB NVMe,Intel Iris Xe Graphics,"14"" Full HD",NaN,NaN,25990000,laptop88,...,i7,1195G7,Core i7-1195G7,4,8,67.3,67.0,77.0,56.0,Intel
314,[Mới 100% Full Box] Laptop GIGABYTE AORUS 15P YD-73S1224GH - Intel Core i7,Core i7-11800H,16GB DDR4,SSD 1024GB NVMe,Nvidia RTX 3080,"15.6"" Full HD 240Hz",NaN,2,58990000,laptop88,...,i7,11800H,Intel® Core i7-11800H,8,16,87.9,88.0,90.0,86.0,Intel
315,[Mới 100% Full Box] Laptop Alienware m15 R6 P109F001BBL - Intel Core i7,Core i7 - 11800H,32GB DDR4,SSD 1024GB NVMe,RTX 3060 6GDDR6,"15.6"" Full HD 165Hz",86,2,58990000,laptop88,...,i7,11800H,Intel® Core i7-11800H,8,16,87.9,88.0,90.0,86.0,Intel


#### Merge the gpu of the laptops with the corresponding score 

In [453]:
df_gpu["gpu"] = df_gpu["gpu"].str.lower()
df_gpu = df_gpu.drop_duplicates(subset = "gpu",ignore_index = True)

In [454]:
df_gpu["gpu"] = df_gpu["gpu"].str.replace("®", "").str.replace("™", "")

In [455]:
with pd.option_context('display.max_rows', 10):
    print(df_gpu)

                                                  gpu  gpu_score
0                                                   0       0.00
1                          nvidia geforce rtx 3050 ti      58.50
2                              intel iris xe graphics       8.34
3                             nvidia geforce rtx 3050      53.70
4                                 amd radeon graphics       3.59
..                                                ...        ...
187           nvidia geforce rtx 3080 max-q 8gb gddr6     128.00
188              nvidia geforce rtx 3080ti 16gb gddr6     139.00
189       intel iris xe graphics eligible, rtx30080ti     139.00
190  intel iris xe graphics, nvidia geforce rtx3080ti     139.00
191  nvidia geforce rtx 3080 ti laptop gpu 16gb gddr6     139.00

[192 rows x 2 columns]


In [456]:
def bs_replace(x):
    try:
        return x.replace(u'\xa0', u' ')
    except:
        return x

In [457]:
df_laptop88["gpu"] = df_laptop88["gpu"].str.lower()
df_laptop88["gpu"] = df_laptop88["gpu"].apply(bs_replace)
df_laptop88["gpu"] = df_laptop88["gpu"].str.replace("®", "").str.replace("™", "")
df_laptop88["gpu"] = df_laptop88["gpu"].str.split(r" |-|/|\|")

In [459]:
df_laptop88["gpu_score"] = np.nan

def gpu_process(df_laptop88):
    for index_gpu, i in enumerate(df_laptop88["gpu"]):
        for index, a_string in enumerate(df_gpu["gpu"]):
            try:
                if all(_ in a_string for _ in i):
                    df_laptop88.loc[index_gpu, "gpu_score"] = df_gpu["gpu_score"].iloc[index]
         
            except:
                df_laptop88.loc[index_gpu, "gpu_score"] = np.nan
                
gpu_process(df_laptop88)

In [460]:
df_laptop88 = df_laptop88.reset_index(drop = True)
df_laptop88.head()

,name,cpu,ram,memory,gpu,monitor,battery,weight,price,website,...,ver,cpu_name,number_of_core,number_of_thread,avg_bench,gamming,desktop,workstation,brand_y,gpu_score
0,[Mới 100% Full Box] Laptop Lenovo Ideapad Gaming 3 15ACH6 82K201XCUS - AMD Ryzen 5 - 5600H | RTX 3050 Ti | 15.6 Inch Full HD 120Hz,R5 - 5600H,8GB DDR4,SSD 256GB NVMe,"[nvidia, rtx, 3050, ti]","15.6"" FHD 120Hz",45,NaN,16790000,laptop88,...,5600H,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,58.50
1,[Mới 100% Full Box] Laptop Gaming GIGABYTE G5 MD-51US113SO - Intel Core i5-11400H | RTX 3050Ti | 144Hz,i5 - 11400H,8GB DDR4,SSD 512GB NMVe,"[nvidia, rtx, 3050ti]","15.6"" Full HD 144Hz",48.96,2,17690000,laptop88,...,11400H,Intel Core i5 11400H,6,12,74.5,74.0,78.0,69.0,Intel,58.50
2,[Mới 99% Full-Box] Laptop Acer Nitro 5 AN515-58-5046 - Intel Core i5-12500H | RAM 16GB | Nvidia RTX 3050Ti | 15.6 Inch Full HD 144Hz,i5 - 12500H,16GB DDR4,SSD 512GB,"[nvidia, rtx, 3050ti]","15.6"" Full HD 144Hz",NaN,2,20490000,laptop88,...,12500H,Intel Core i5-12500H,12,16,94.8,95.0,97.0,94.0,Intel,58.50
3,[Mới 99% Full-Box] Laptop Acer Nitro 5 Eagle AN515-57-536Q / 56FC - Intel Core i5 11400H GTX 1650,i5 - 11400H,8GB DDR4,SSD 256GB,"[nvidia, gtx, 1650]","15.6"" Full HD 144Hz",NaN,2,15590000,laptop88,...,11400H,Intel Core i5 11400H,6,12,74.5,74.0,78.0,69.0,Intel,41.00
4,[Mới 100% Full Box] Laptop Lenovo IdeaPad 5 Pro 16ACH6 82L500F5US - AMD Ryzen 5-5600H | 16 Inch WQXGA 100% sRGB,AMD R5-5600H,8GB DDR4,SSD 512GB,"[amd, radeon, vega]","16"" WQXGA 100% sRGB",75,NaN,15690000,laptop88,...,5600H,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.98


In [461]:
df_laptop88.columns

Index(['name', 'cpu', 'ram', 'memory', 'gpu', 'monitor', 'battery', 'weight',
       'price', 'website', 'monitor_size', 'ram_capacity', 'ram_type',
       'HDD_capacity', 'SSD_capacity', 'brand_x', 'level', 'ver', 'cpu_name',
       'number_of_core', 'number_of_thread', 'avg_bench', 'gamming', 'desktop',
       'workstation', 'brand_y', 'gpu_score'],
      dtype='object')

#### Rename the columns

In [462]:
df_laptop88.rename(columns = {"number_of_core":"cpu_core_num", "number_of_thread":"cpu_thread_num"}, inplace = True)

In [463]:
df_laptop88 = df_laptop88.drop(columns = ["monitor","level", "ver", "cpu_name", "brand_x", "brand_y"])
df_laptop88

,name,cpu,ram,memory,gpu,battery,weight,price,website,monitor_size,...,ram_type,HDD_capacity,SSD_capacity,cpu_core_num,cpu_thread_num,avg_bench,gamming,desktop,workstation,gpu_score
0,[Mới 100% Full Box] Laptop Lenovo Ideapad Gaming 3 15ACH6 82K201XCUS - AMD Ryzen 5 - 5600H | RTX 3050 Ti | 15.6 Inch Full HD 120Hz,R5 - 5600H,8GB DDR4,SSD 256GB NVMe,"[nvidia, rtx, 3050, ti]",45,NaN,16790000,laptop88,15.6,...,DDR4,0,256,NaN,NaN,NaN,NaN,NaN,NaN,58.50
1,[Mới 100% Full Box] Laptop Gaming GIGABYTE G5 MD-51US113SO - Intel Core i5-11400H | RTX 3050Ti | 144Hz,i5 - 11400H,8GB DDR4,SSD 512GB NMVe,"[nvidia, rtx, 3050ti]",48.96,2,17690000,laptop88,15.6,...,DDR4,0,512,6,12,74.5,74.0,78.0,69.0,58.50
2,[Mới 99% Full-Box] Laptop Acer Nitro 5 AN515-58-5046 - Intel Core i5-12500H | RAM 16GB | Nvidia RTX 3050Ti | 15.6 Inch Full HD 144Hz,i5 - 12500H,16GB DDR4,SSD 512GB,"[nvidia, rtx, 3050ti]",NaN,2,20490000,laptop88,15.6,...,DDR4,0,512,12,16,94.8,95.0,97.0,94.0,58.50
3,[Mới 99% Full-Box] Laptop Acer Nitro 5 Eagle AN515-57-536Q / 56FC - Intel Core i5 11400H GTX 1650,i5 - 11400H,8GB DDR4,SSD 256GB,"[nvidia, gtx, 1650]",NaN,2,15590000,laptop88,15.6,...,DDR4,0,256,6,12,74.5,74.0,78.0,69.0,41.00
4,[Mới 100% Full Box] Laptop Lenovo IdeaPad 5 Pro 16ACH6 82L500F5US - AMD Ryzen 5-5600H | 16 Inch WQXGA 100% sRGB,AMD R5-5600H,8GB DDR4,SSD 512GB,"[amd, radeon, vega]",75,NaN,15690000,laptop88,16,...,DDR4,0,512,NaN,NaN,NaN,NaN,NaN,NaN,8.98
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
312,[Mới 100% Full Box] Laptop Gaming Dell Alienware M15 R5 Ryzen Edition - R9 5900HX RTX 3070,AMD R9-5900HX,16GB DDR4,SSD 1024GB NVMe,"[nvidia, rtx, 3070]",NaN,2,54990000,laptop88,15.6,...,DDR4,0,1024,NaN,NaN,NaN,NaN,NaN,NaN,112.00
313,[Mới 100% Full Box] Laptop Gigabyte U4 UD 70S1823SO - Intel Core i7,Core i7-1195G7,16GB DDR4,SSD 512GB NVMe,"[intel, iris, xe, graphics]",NaN,NaN,25990000,laptop88,14,...,DDR4,0,512,4,8,67.3,67.0,77.0,56.0,8.34
314,[Mới 100% Full Box] Laptop GIGABYTE AORUS 15P YD-73S1224GH - Intel Core i7,Core i7-11800H,16GB DDR4,SSD 1024GB NVMe,"[nvidia, rtx, 3080]",NaN,2,58990000,laptop88,15.6,...,DDR4,0,1024,8,16,87.9,88.0,90.0,86.0,128.00
315,[Mới 100% Full Box] Laptop Alienware m15 R6 P109F001BBL - Intel Core i7,Core i7 - 11800H,32GB DDR4,SSD 1024GB NVMe,"[rtx, 3060, 6gddr6]",86,2,58990000,laptop88,15.6,...,DDR4,0,1024,8,16,87.9,88.0,90.0,86.0,NaN


In [464]:
df_laptop88.isnull().sum()

name                0
cpu                 0
ram                 0
memory              0
gpu                 0
battery           147
weight            126
price               0
website             0
monitor_size        0
ram_capacity        0
ram_type           15
HDD_capacity        0
SSD_capacity        0
cpu_core_num      107
cpu_thread_num    107
avg_bench         107
gamming           107
desktop           107
workstation       107
gpu_score          40
dtype: int64

## Now, we move on to the thinkpro website

In [468]:
df.head()

,Unnamed: 0,name,cpu,ram,monitor,gpu,memory,battery,connection,weight,price,website
0,0,0,0,0,0,0,0,10WHr,0,0,0,thinkpro
1,748,Lenovo ThinkPad E15 Gen 4,0,0,0,0,0,10WHr,0,0,0,thinkpro
2,749,Lenovo IdeaPad Gaming 3i Gen 7,0,0,0,0,0,10WHr,0,0,0,thinkpro
3,750,Lenovo ThinkPad P15v Gen 3,0,0,0,0,0,10WHr,0,0,0,thinkpro
4,751,Lenovo Legion S7 Gen 7,0,0,0,0,0,10WHr,0,0,0,thinkpro


In [469]:
df.shape

(646, 12)

### Remove duplicates

In [470]:
df = df.drop_duplicates(ignore_index = True)
del df[df.columns[0]]

In [471]:
df.shape

(646, 11)

In [472]:
df.dtypes

name          object
cpu           object
ram           object
monitor       object
gpu           object
memory        object
battery       object
connection    object
weight        object
price          int64
website       object
dtype: object

In [473]:
df["battery"] = df["battery"].replace("10WHr", "0")

### Transform *price*, *battery*, *weight* to the right data type

In [474]:
df["battery"]

0            0
1            0
2            0
3            0
4            0
        ...   
641      80WHr
642    99.9WHr
643      80WHr
644      82WHr
645      99WHr
Name: battery, Length: 646, dtype: object

In [475]:
df["battery"] = df["battery"].str.replace("WHr", "")

In [476]:
df["battery"] = df["battery"].astype("float")

In [477]:
df["weight"] = df["weight"].str.replace("kg", "")

In [478]:
df["weight"] = df["weight"].astype("float")

### Dropping the rows with missing values in the *price* columns and the rows with more than 5 features

In [479]:
df = df.replace(["0", 0], np.nan)

In [480]:
df = df.dropna(subset = ['price'], how = "any")
df = df.dropna(thresh = 5)
df = df.reset_index(drop=True)
df

,name,cpu,ram,monitor,gpu,memory,battery,connection,weight,price,website
0,Sạc Macbook Hyperjuice Ultimate Charger 110W DUAL WIRELESS CHARGER (HJ-UC5-W15LR),"Intel Core i5 1235U, 10 nhân, 12 luồng","8GB, DDR4, 3200 MHz","14"", 1920 x 1080 px, IPS, 65% sRGB, 60 Hz",Intel Iris Xe Graphics,SSD 512GB,39.3,"1 x Type-C, 1 x USB-A",1.400,3590000.0,thinkpro
1,Ghế Công Thái Học HyperWork HW01 (Arm 3D - Gray),"Intel Core i5 11300H, 4 nhân, 8 luồng","16GB, LPDDR4X, 4266 MHz","14.4"", 2400 x 1600 px, PixelSense Flow Display, Cảm ứng, 100% sRGB, 120 Hz",Intel Iris Xe Graphics,SSD 512GB,58.0,2 x Type-C,1.743,3790000.0,thinkpro
2,Màn hình chuyên đồ hoạ ViewSonic VG2755-2K (QHD/IPS/60Hz),"AMD Ryzen R5-5625U, 6 nhân, 12 luồng","8GB, DDR4, 3200 MHz","15.6"", 1920 x 1080 px, IPS, Chống chói, 65% sRGB",AMD Radeon Graphics,SSD 512GB,39.3,"1 x Type-C, 1 x USB-A",1.700,7790000.0,thinkpro
3,Asus Vivobook X515 (Chính Hãng) (X515MA-BR481T),"Intel Celeron N4020, 2 nhân, 2 luồng","4GB, DDR4, 3200 MHz","15.6"", 1366 x 768 px, TN, Matte, 60 Hz",Intel UHD Graphics,"SSD 256GB, HDD 1024GB",37.0,"1 x Type-C, 3 x USB-A",NaN,7990000.0,thinkpro
4,ASUS Vivobook X515JA,"Intel Core i3 1005G1, 2 nhân, 4 luồng","8GB, 3200 MHz","15.6"", 1366 x 768 px, TN, Chống chói, 45% sRGB",Intel UHD Graphics,SSD 256GB,37.0,"1 x Type-C, 2 x USB-A",1.800,8990000.0,thinkpro
...,...,...,...,...,...,...,...,...,...,...,...
556,Razer Blade 15 Advanced (Intel Gen 11) (RZ09-0409CEC3-R3U1),"Intel Core i9 11900H, 8 nhân, 16 luồng","32GB, DDR4, 3200 MHz","15.6"", 3840 x 2160 px, OLED, Glossy, Cảm ứng, 60 Hz",Intel UHD,"SSD 1024GB, HDD 0GB",80.0,"2 x Type-C, 3 x USB-A, Thunderbolt",2.010,114990000.0,thinkpro
557,MSI Stealth GS77 2022,"Intel Core i9-12900H, 16 nhân, 24 luồng","32GB, DDR5, 4800 MHz","17.3"", 3840x2160 px, IPS, Chống chói, 120 Hz",Nvidia Geforce RTX 3080Ti 16GB GDDR6,SSD 2TB,99.9,"2 x Type-C, 2 x USB-A, Thunderbolt",2.800,119990000.0,thinkpro
558,Razer Blade 15 2022,"Intel Core i9 12900H, 14 nhân, 20 luồng","32GB, DDR5, 4800 MHz","15.6"", 3840 x 2160 px, IPS, 100% sRGB, 144 Hz","Intel® Iris® Xe Graphics eligible, RTX30080Ti",SSD 1024GB,80.0,"1 x Type-C, 2 x USB-A",2.010,129990000.0,thinkpro
559,Razer Blade 17 2022,"Intel Core i9 12900H, 14 nhân, 20 luồng","32GB, 4800 MHz","17.3"", 3840 x 2160 px, IPS, 100% sRGB, 144 Hz","Intel Iris Xe Graphics, Nvidia Geforce RTX3080Ti",SSD 1024GB,82.0,"2 x Type-C, 3 x USB-A, Thunderbolt",2.750,129990000.0,thinkpro


### Check the number of missing values

In [481]:
df.isnull().sum()

name           0
cpu            0
ram            1
monitor        1
gpu            4
memory         1
battery       16
connection     1
weight        56
price          0
website        0
dtype: int64

### Add new feature: the laptop brand

In [482]:
brand_dict = {"Lenovo": ["Lenovo", "ThinkPad", "Thinkpad"],
              "Apple": ["Apple"],
              "HP": ["HP", "Hp"],
              "Dell": ["Dell", "New Inspiron", "Alienware"],
              "Asus": ["Asus", "ASUS"],
              "Acer": ["Acer"],
              "Microsoft":["Microsoft", "Surface"],
              "Huawei": ["Huawei", "HUAWEI"],
              "Xiaomi": ["Xiaomi"],
              "LG": ["LG"],
              "GIGABYTE": ["GIGABYTE"],
              "Samsung": ["Samsung"],
              "Razer": ["Razer"],
              "MSI": ["MSI"],
              "AVITA": ["AVITA"],
              "Colorful": ["Colorful"]}

In [483]:
def get_brand(brand_name, brand_dict = brand_dict):
    for i in brand_dict.keys():
        for j in brand_dict[i]:
            if j in brand_name:
                return i
    return "Other"

In [484]:
df["laptop_brand"] = df["name"].apply(get_brand)
df_laptop88["laptop_brand"] = df_laptop88["name"].apply(get_brand)

In [485]:
df.head()

,name,cpu,ram,monitor,gpu,memory,battery,connection,weight,price,website,laptop_brand
0,Sạc Macbook Hyperjuice Ultimate Charger 110W DUAL WIRELESS CHARGER (HJ-UC5-W15LR),"Intel Core i5 1235U, 10 nhân, 12 luồng","8GB, DDR4, 3200 MHz","14"", 1920 x 1080 px, IPS, 65% sRGB, 60 Hz",Intel Iris Xe Graphics,SSD 512GB,39.3,"1 x Type-C, 1 x USB-A",1.400,3590000.0,thinkpro,Other
1,Ghế Công Thái Học HyperWork HW01 (Arm 3D - Gray),"Intel Core i5 11300H, 4 nhân, 8 luồng","16GB, LPDDR4X, 4266 MHz","14.4"", 2400 x 1600 px, PixelSense Flow Display, Cảm ứng, 100% sRGB, 120 Hz",Intel Iris Xe Graphics,SSD 512GB,58.0,2 x Type-C,1.743,3790000.0,thinkpro,Other
2,Màn hình chuyên đồ hoạ ViewSonic VG2755-2K (QHD/IPS/60Hz),"AMD Ryzen R5-5625U, 6 nhân, 12 luồng","8GB, DDR4, 3200 MHz","15.6"", 1920 x 1080 px, IPS, Chống chói, 65% sRGB",AMD Radeon Graphics,SSD 512GB,39.3,"1 x Type-C, 1 x USB-A",1.700,7790000.0,thinkpro,Other
3,Asus Vivobook X515 (Chính Hãng) (X515MA-BR481T),"Intel Celeron N4020, 2 nhân, 2 luồng","4GB, DDR4, 3200 MHz","15.6"", 1366 x 768 px, TN, Matte, 60 Hz",Intel UHD Graphics,"SSD 256GB, HDD 1024GB",37.0,"1 x Type-C, 3 x USB-A",NaN,7990000.0,thinkpro,Asus
4,ASUS Vivobook X515JA,"Intel Core i3 1005G1, 2 nhân, 4 luồng","8GB, 3200 MHz","15.6"", 1366 x 768 px, TN, Chống chói, 45% sRGB",Intel UHD Graphics,SSD 256GB,37.0,"1 x Type-C, 2 x USB-A",1.800,8990000.0,thinkpro,Asus


### Split the monitor column into its *size*

In [486]:
monitor = df["monitor"].str.split(", ").to_frame()

In [487]:
monitor

,monitor
0,"[14"", 1920 x 1080 px, IPS, 65% sRGB, 60 Hz]"
1,"[14.4"", 2400 x 1600 px, PixelSense Flow Display, Cảm ứng, 100% sRGB, 120 Hz]"
2,"[15.6"", 1920 x 1080 px, IPS, Chống chói, 65% sRGB]"
3,"[15.6"", 1366 x 768 px, TN, Matte, 60 Hz]"
4,"[15.6"", 1366 x 768 px, TN, Chống chói, 45% sRGB]"
...,...
556,"[15.6"", 3840 x 2160 px, OLED, Glossy, Cảm ứng, 60 Hz]"
557,"[17.3"", 3840x2160 px, IPS, Chống chói, 120 Hz]"
558,"[15.6"", 3840 x 2160 px, IPS, 100% sRGB, 144 Hz]"
559,"[17.3"", 3840 x 2160 px, IPS, 100% sRGB, 144 Hz]"


In [488]:
def size(x):
    try:
        for i in x:
            if "\"" in i:
                return i.replace("\"", "")
    except:
        return x


In [489]:
df["monitor_size"] = monitor["monitor"].apply(size)

In [490]:
df.tail(2)

,name,cpu,ram,monitor,gpu,memory,battery,connection,weight,price,website,laptop_brand,monitor_size
559,Razer Blade 17 2022,"Intel Core i9 12900H, 14 nhân, 20 luồng","32GB, 4800 MHz","17.3"", 3840 x 2160 px, IPS, 100% sRGB, 144 Hz","Intel Iris Xe Graphics, Nvidia Geforce RTX3080Ti",SSD 1024GB,82.0,"2 x Type-C, 3 x USB-A, Thunderbolt",2.75,129990000.0,thinkpro,Razer,17.3
560,MSI Titan GT77 2022 (12UHS),"Intel Core i9-12900HX Processor, 16 nhân, 24 luồng","64GB, DDR5, 4800 MHz","17.3"", 3840x2160 px, IPS, Chống chói, 120 Hz",NVIDIA GeForce RTX 3080 Ti Laptop GPU 16GB GDDR6,SSD 2TB,99.0,"2 x Type-C, 3 x USB-A, Thunderbolt",3.30,139990000.0,thinkpro,MSI,17.3


In [491]:
ram = df["ram"].str.split(", ").to_frame()
ram.iloc[2,0]

['8GB', 'DDR4', '3200 MHz']

In [492]:
def capacity(x):
    try: 
        for i in x:
            if "GB" in i:
                return i.replace("GB", "")
        return np.nan
    except:
        return x

def ram_type(x):
    try: 
        for i in x:
            if i == "DDR4":
                return i
            if "DDR5" in i:
                return i
            if i == "LPDDR5":
                return i
            if i == "LPDDR4x":
                return i
            if i == "LPDDR3L":
                return i
            if i == "LPDDR3":
                return i
            if i == "DDR3L":
                return i
        return np.nan
    except:
        return x

In [493]:
df["ram_capacity"] = ram["ram"].apply(capacity)
df["ram_type"] = ram["ram"].apply(ram_type)


In [494]:
df.head()

,name,cpu,ram,monitor,gpu,memory,battery,connection,weight,price,website,laptop_brand,monitor_size,ram_capacity,ram_type
0,Sạc Macbook Hyperjuice Ultimate Charger 110W DUAL WIRELESS CHARGER (HJ-UC5-W15LR),"Intel Core i5 1235U, 10 nhân, 12 luồng","8GB, DDR4, 3200 MHz","14"", 1920 x 1080 px, IPS, 65% sRGB, 60 Hz",Intel Iris Xe Graphics,SSD 512GB,39.3,"1 x Type-C, 1 x USB-A",1.400,3590000.0,thinkpro,Other,14,8,DDR4
1,Ghế Công Thái Học HyperWork HW01 (Arm 3D - Gray),"Intel Core i5 11300H, 4 nhân, 8 luồng","16GB, LPDDR4X, 4266 MHz","14.4"", 2400 x 1600 px, PixelSense Flow Display, Cảm ứng, 100% sRGB, 120 Hz",Intel Iris Xe Graphics,SSD 512GB,58.0,2 x Type-C,1.743,3790000.0,thinkpro,Other,14.4,16,NaN
2,Màn hình chuyên đồ hoạ ViewSonic VG2755-2K (QHD/IPS/60Hz),"AMD Ryzen R5-5625U, 6 nhân, 12 luồng","8GB, DDR4, 3200 MHz","15.6"", 1920 x 1080 px, IPS, Chống chói, 65% sRGB",AMD Radeon Graphics,SSD 512GB,39.3,"1 x Type-C, 1 x USB-A",1.700,7790000.0,thinkpro,Other,15.6,8,DDR4
3,Asus Vivobook X515 (Chính Hãng) (X515MA-BR481T),"Intel Celeron N4020, 2 nhân, 2 luồng","4GB, DDR4, 3200 MHz","15.6"", 1366 x 768 px, TN, Matte, 60 Hz",Intel UHD Graphics,"SSD 256GB, HDD 1024GB",37.0,"1 x Type-C, 3 x USB-A",NaN,7990000.0,thinkpro,Asus,15.6,4,DDR4
4,ASUS Vivobook X515JA,"Intel Core i3 1005G1, 2 nhân, 4 luồng","8GB, 3200 MHz","15.6"", 1366 x 768 px, TN, Chống chói, 45% sRGB",Intel UHD Graphics,SSD 256GB,37.0,"1 x Type-C, 2 x USB-A",1.800,8990000.0,thinkpro,Asus,15.6,8,NaN


### Replace the memory feature by 2 different features:
- HDD_capacity: 
- SSD_capacity:

In [495]:
df["memory"].value_counts()

SSD 512GB                268
SSD 256GB                109
SSD 512GB, HDD 0GB        57
SSD 1024GB                49
SSD 256GB, HDD 0GB        23
SSD 1024GB, HDD 0GB       18
SSD 128GB                 10
SSD 2TB                    9
SSD 512GB, HDD 1GB         6
SSD 2TB, HDD 0GB           3
SSD 1000GB, HDD 0GB        2
SSD 1.001953125TB          1
HDD 1024GB                 1
SSD 4TB                    1
SSD 256GB, HDD 2TB         1
SSD 256GB, HDD 1024GB      1
SSD 1.9765625TB            1
Name: memory, dtype: int64

In [496]:
df["memory"] = df["memory"].str.replace("HDD 0GB", "").str.strip(" , ")
df["memory"] = df["memory"].str.replace("1TB", "1024GB")
df["memory"] = df["memory"].str.replace("2TB", "2048GB")
df["memory"] = df["memory"].str.replace("1.9765625TB", "2048GB")
df["memory"] = df["memory"].str.replace("1.001953125TB", "1024GB")
df["memory"] = df["memory"].str.replace("4TB", "4096GB")

df.loc[df["memory"] == "","memory"] = np.nan

<ipython-input-496-5636d94ae61a>:4: FutureWarning: The default value of regex will change from True to False in a future version.
  df["memory"] = df["memory"].str.replace("1.9765625TB", "2048GB")
<ipython-input-496-5636d94ae61a>:5: FutureWarning: The default value of regex will change from True to False in a future version.
  df["memory"] = df["memory"].str.replace("1.001953125TB", "1024GB")


In [497]:
df["memory"].value_counts()

SSD 512GB                325
SSD 256GB                132
SSD 1024GB                68
SSD 2048GB                13
SSD 128GB                 10
SSD 512GB, HDD 1GB         6
SSD 1000GB                 2
SSD 4096GB                 1
SSD 256GB, HDD 1024GB      1
SSD 256GB, HDD 2048GB      1
HDD 1024GB                 1
Name: memory, dtype: int64

In [498]:
memory= df["memory"].str.split(",").to_frame()
memory

,memory
0,[SSD 512GB]
1,[SSD 512GB]
2,[SSD 512GB]
3,"[SSD 256GB, HDD 1024GB]"
4,[SSD 256GB]
...,...
556,[SSD 1024GB]
557,[SSD 2048GB]
558,[SSD 1024GB]
559,[SSD 1024GB]


In [499]:
def hdd_capacity(x):
    try:
        for i in x:
          if "HDD" in i:
             return re.sub(r"\D", "", i)
        return 0
    except:
        return x

def ssd_capacity(x):
    try:
        for i in x:
          if "SSD" in i:
             return re.sub(r"\D", "", i)
        return 0
    except:
        return x

In [500]:
df["HDD_capacity"] = memory["memory"].apply(hdd_capacity)
df["SSD_capacity"] = memory["memory"].apply(ssd_capacity)

In [501]:
df.tail()

,name,cpu,ram,monitor,gpu,memory,battery,connection,weight,price,website,laptop_brand,monitor_size,ram_capacity,ram_type,HDD_capacity,SSD_capacity
556,Razer Blade 15 Advanced (Intel Gen 11) (RZ09-0409CEC3-R3U1),"Intel Core i9 11900H, 8 nhân, 16 luồng","32GB, DDR4, 3200 MHz","15.6"", 3840 x 2160 px, OLED, Glossy, Cảm ứng, 60 Hz",Intel UHD,SSD 1024GB,80.0,"2 x Type-C, 3 x USB-A, Thunderbolt",2.01,114990000.0,thinkpro,Razer,15.6,32,DDR4,0,1024
557,MSI Stealth GS77 2022,"Intel Core i9-12900H, 16 nhân, 24 luồng","32GB, DDR5, 4800 MHz","17.3"", 3840x2160 px, IPS, Chống chói, 120 Hz",Nvidia Geforce RTX 3080Ti 16GB GDDR6,SSD 2048GB,99.9,"2 x Type-C, 2 x USB-A, Thunderbolt",2.80,119990000.0,thinkpro,MSI,17.3,32,DDR5,0,2048
558,Razer Blade 15 2022,"Intel Core i9 12900H, 14 nhân, 20 luồng","32GB, DDR5, 4800 MHz","15.6"", 3840 x 2160 px, IPS, 100% sRGB, 144 Hz","Intel® Iris® Xe Graphics eligible, RTX30080Ti",SSD 1024GB,80.0,"1 x Type-C, 2 x USB-A",2.01,129990000.0,thinkpro,Razer,15.6,32,DDR5,0,1024
559,Razer Blade 17 2022,"Intel Core i9 12900H, 14 nhân, 20 luồng","32GB, 4800 MHz","17.3"", 3840 x 2160 px, IPS, 100% sRGB, 144 Hz","Intel Iris Xe Graphics, Nvidia Geforce RTX3080Ti",SSD 1024GB,82.0,"2 x Type-C, 3 x USB-A, Thunderbolt",2.75,129990000.0,thinkpro,Razer,17.3,32,NaN,0,1024
560,MSI Titan GT77 2022 (12UHS),"Intel Core i9-12900HX Processor, 16 nhân, 24 luồng","64GB, DDR5, 4800 MHz","17.3"", 3840x2160 px, IPS, Chống chói, 120 Hz",NVIDIA GeForce RTX 3080 Ti Laptop GPU 16GB GDDR6,SSD 2048GB,99.0,"2 x Type-C, 3 x USB-A, Thunderbolt",3.30,139990000.0,thinkpro,MSI,17.3,64,DDR5,0,2048


#### Merge the cpu of the laptops with the corresponding score

In [502]:
with pd.option_context('display.max_rows', 10,):
   print(df["cpu"].value_counts())


Intel Core i5 1135G7, 4 nhân, 8 luồng                 38
Intel Core i7 11800H, 8 nhân, 16 luồng                23
Intel Core i7 1165G7, 4 nhân, 8 luồng                 22
Intel Core i5-1135G7, 4 nhân, 8 luồng                 17
AMD Ryzen 5 5600H, 6 nhân, 12 luồng                   17
                                                      ..
Intel Core i7-1250U, 10 nhân, 12 luồng                 1
AMD Ryzen 7-5700U, 8 nhân, 16 luồng                    1
Intel® Core™ i7-11375H, 4 nhân, 8 luồng                1
Intel  Core i9 12900H, 14 nhân, 20 luồng               1
Intel Core i9-12900HX Processor, 16 nhân, 24 luồng     1
Name: cpu, Length: 196, dtype: int64


In [503]:
cpu_brand_dict = {"Intel":["Intel","Core"],
                  "AMD": ["Ryzen", "AMD"],
                  "Apple":["Apple"]}

In [504]:
def get_cpu_brand(brand_name, cpu_brand_dict = cpu_brand_dict):
  for i in cpu_brand_dict.keys():
    for j in cpu_brand_dict[i]:
      if j in brand_name:
        return i
  return "Other"

In [505]:
df["brand"] = df["cpu"].apply(get_cpu_brand)
df_laptop88["cpu_brand"] = df["cpu"].apply(get_cpu_brand)

In [506]:
cpu = df["cpu"].str.split(", ")
cpu.head()

0    [Intel Core i5 1235U, 10 nhân, 12 luồng]
1     [Intel Core i5 11300H, 4 nhân, 8 luồng]
2      [AMD Ryzen R5-5625U, 6 nhân, 12 luồng]
3      [Intel Celeron N4020, 2 nhân, 2 luồng]
4     [Intel Core i3 1005G1, 2 nhân, 4 luồng]
Name: cpu, dtype: object

In [507]:
df[["cpu_name", "cpu_core_num", "cpu_thread_num"]] = cpu.apply(pd.Series).rename(columns = {0:"cpu_name", 1: "cpu_core_num", 2: "cpu_thread_num"})

In [508]:
df["cpu_core_num"] = df["cpu_core_num"].str.replace(" nhân", "").astype(float)
df["cpu_thread_num"] = df["cpu_thread_num"].str.replace(" luồng", "").astype(float)

In [509]:
df["cpu_name"] = df["cpu_name"].apply(bs_replace)

In [510]:
df["level"] = df["cpu_name"].apply(get_cpu_level)
df["ver"] = df["cpu_name"].apply(get_cpu_ver)

In [511]:
df

,name,cpu,ram,monitor,gpu,memory,battery,connection,weight,price,...,ram_capacity,ram_type,HDD_capacity,SSD_capacity,brand,cpu_name,cpu_core_num,cpu_thread_num,level,ver
0,Sạc Macbook Hyperjuice Ultimate Charger 110W DUAL WIRELESS CHARGER (HJ-UC5-W15LR),"Intel Core i5 1235U, 10 nhân, 12 luồng","8GB, DDR4, 3200 MHz","14"", 1920 x 1080 px, IPS, 65% sRGB, 60 Hz",Intel Iris Xe Graphics,SSD 512GB,39.3,"1 x Type-C, 1 x USB-A",1.400,3590000.0,...,8,DDR4,0,512,Intel,Intel Core i5 1235U,10.0,12.0,i5,1235U
1,Ghế Công Thái Học HyperWork HW01 (Arm 3D - Gray),"Intel Core i5 11300H, 4 nhân, 8 luồng","16GB, LPDDR4X, 4266 MHz","14.4"", 2400 x 1600 px, PixelSense Flow Display, Cảm ứng, 100% sRGB, 120 Hz",Intel Iris Xe Graphics,SSD 512GB,58.0,2 x Type-C,1.743,3790000.0,...,16,NaN,0,512,Intel,Intel Core i5 11300H,4.0,8.0,i5,11300H
2,Màn hình chuyên đồ hoạ ViewSonic VG2755-2K (QHD/IPS/60Hz),"AMD Ryzen R5-5625U, 6 nhân, 12 luồng","8GB, DDR4, 3200 MHz","15.6"", 1920 x 1080 px, IPS, Chống chói, 65% sRGB",AMD Radeon Graphics,SSD 512GB,39.3,"1 x Type-C, 1 x USB-A",1.700,7790000.0,...,8,DDR4,0,512,AMD,AMD Ryzen R5-5625U,6.0,12.0,R5,5625U
3,Asus Vivobook X515 (Chính Hãng) (X515MA-BR481T),"Intel Celeron N4020, 2 nhân, 2 luồng","4GB, DDR4, 3200 MHz","15.6"", 1366 x 768 px, TN, Matte, 60 Hz",Intel UHD Graphics,"SSD 256GB, HDD 1024GB",37.0,"1 x Type-C, 3 x USB-A",NaN,7990000.0,...,4,DDR4,1024,256,Intel,Intel Celeron N4020,2.0,2.0,NaN,N4020
4,ASUS Vivobook X515JA,"Intel Core i3 1005G1, 2 nhân, 4 luồng","8GB, 3200 MHz","15.6"", 1366 x 768 px, TN, Chống chói, 45% sRGB",Intel UHD Graphics,SSD 256GB,37.0,"1 x Type-C, 2 x USB-A",1.800,8990000.0,...,8,NaN,0,256,Intel,Intel Core i3 1005G1,2.0,4.0,i3,1005G1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
556,Razer Blade 15 Advanced (Intel Gen 11) (RZ09-0409CEC3-R3U1),"Intel Core i9 11900H, 8 nhân, 16 luồng","32GB, DDR4, 3200 MHz","15.6"", 3840 x 2160 px, OLED, Glossy, Cảm ứng, 60 Hz",Intel UHD,SSD 1024GB,80.0,"2 x Type-C, 3 x USB-A, Thunderbolt",2.010,114990000.0,...,32,DDR4,0,1024,Intel,Intel Core i9 11900H,8.0,16.0,i9,11900H
557,MSI Stealth GS77 2022,"Intel Core i9-12900H, 16 nhân, 24 luồng","32GB, DDR5, 4800 MHz","17.3"", 3840x2160 px, IPS, Chống chói, 120 Hz",Nvidia Geforce RTX 3080Ti 16GB GDDR6,SSD 2048GB,99.9,"2 x Type-C, 2 x USB-A, Thunderbolt",2.800,119990000.0,...,32,DDR5,0,2048,Intel,Intel Core i9-12900H,16.0,24.0,i9,12900H
558,Razer Blade 15 2022,"Intel Core i9 12900H, 14 nhân, 20 luồng","32GB, DDR5, 4800 MHz","15.6"", 3840 x 2160 px, IPS, 100% sRGB, 144 Hz","Intel® Iris® Xe Graphics eligible, RTX30080Ti",SSD 1024GB,80.0,"1 x Type-C, 2 x USB-A",2.010,129990000.0,...,32,DDR5,0,1024,Intel,Intel Core i9 12900H,14.0,20.0,i9,12900H
559,Razer Blade 17 2022,"Intel Core i9 12900H, 14 nhân, 20 luồng","32GB, 4800 MHz","17.3"", 3840 x 2160 px, IPS, 100% sRGB, 144 Hz","Intel Iris Xe Graphics, Nvidia Geforce RTX3080Ti",SSD 1024GB,82.0,"2 x Type-C, 3 x USB-A, Thunderbolt",2.750,129990000.0,...,32,NaN,0,1024,Intel,Intel Core i9 12900H,14.0,20.0,i9,12900H


In [512]:
df = df.merge(df_cpu, how = "left", on = ["brand", "level", "ver"])
df

,name,cpu,ram,monitor,gpu,memory,battery,connection,weight,price,...,cpu_thread_num,level,ver,cpu_name_y,number_of_core,number_of_thread,avg_bench,gamming,desktop,workstation
0,Sạc Macbook Hyperjuice Ultimate Charger 110W DUAL WIRELESS CHARGER (HJ-UC5-W15LR),"Intel Core i5 1235U, 10 nhân, 12 luồng","8GB, DDR4, 3200 MHz","14"", 1920 x 1080 px, IPS, 65% sRGB, 60 Hz",Intel Iris Xe Graphics,SSD 512GB,39.3,"1 x Type-C, 1 x USB-A",1.400,3590000.0,...,12.0,i5,1235U,Intel Core i5-1235U,10,12,63.6,64.0,72.0,59.0
1,Ghế Công Thái Học HyperWork HW01 (Arm 3D - Gray),"Intel Core i5 11300H, 4 nhân, 8 luồng","16GB, LPDDR4X, 4266 MHz","14.4"", 2400 x 1600 px, PixelSense Flow Display, Cảm ứng, 100% sRGB, 120 Hz",Intel Iris Xe Graphics,SSD 512GB,58.0,2 x Type-C,1.743,3790000.0,...,8.0,i5,11300H,Intel Core i5 11300H,4,8,66.4,66.0,73.0,57.0
2,Màn hình chuyên đồ hoạ ViewSonic VG2755-2K (QHD/IPS/60Hz),"AMD Ryzen R5-5625U, 6 nhân, 12 luồng","8GB, DDR4, 3200 MHz","15.6"", 1920 x 1080 px, IPS, Chống chói, 65% sRGB",AMD Radeon Graphics,SSD 512GB,39.3,"1 x Type-C, 1 x USB-A",1.700,7790000.0,...,12.0,R5,5625U,AMD Ryzen R5-5625U,6,12,0.0,0.0,0.0,0.0
3,Asus Vivobook X515 (Chính Hãng) (X515MA-BR481T),"Intel Celeron N4020, 2 nhân, 2 luồng","4GB, DDR4, 3200 MHz","15.6"", 1366 x 768 px, TN, Matte, 60 Hz",Intel UHD Graphics,"SSD 256GB, HDD 1024GB",37.0,"1 x Type-C, 3 x USB-A",NaN,7990000.0,...,2.0,NaN,N4020,Intel Celeron N4020,2,2,20.9,21.0,27.0,15.0
4,ASUS Vivobook X515JA,"Intel Core i3 1005G1, 2 nhân, 4 luồng","8GB, 3200 MHz","15.6"", 1366 x 768 px, TN, Chống chói, 45% sRGB",Intel UHD Graphics,SSD 256GB,37.0,"1 x Type-C, 2 x USB-A",1.800,8990000.0,...,4.0,i3,1005G1,Intel Core i3 1005G1,2,4,90.6,91.0,88.0,75.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
556,Razer Blade 15 Advanced (Intel Gen 11) (RZ09-0409CEC3-R3U1),"Intel Core i9 11900H, 8 nhân, 16 luồng","32GB, DDR4, 3200 MHz","15.6"", 3840 x 2160 px, OLED, Glossy, Cảm ứng, 60 Hz",Intel UHD,SSD 1024GB,80.0,"2 x Type-C, 3 x USB-A, Thunderbolt",2.010,114990000.0,...,16.0,i9,11900H,Intel Core i9 11900H,8,16,90.7,91.0,92.0,91.0
557,MSI Stealth GS77 2022,"Intel Core i9-12900H, 16 nhân, 24 luồng","32GB, DDR5, 4800 MHz","17.3"", 3840x2160 px, IPS, Chống chói, 120 Hz",Nvidia Geforce RTX 3080Ti 16GB GDDR6,SSD 2048GB,99.9,"2 x Type-C, 2 x USB-A, Thunderbolt",2.800,119990000.0,...,24.0,i9,12900H,Intel Core i9-12900H,14,20,100.0,100.0,103.0,106.0
558,Razer Blade 15 2022,"Intel Core i9 12900H, 14 nhân, 20 luồng","32GB, DDR5, 4800 MHz","15.6"", 3840 x 2160 px, IPS, 100% sRGB, 144 Hz","Intel® Iris® Xe Graphics eligible, RTX30080Ti",SSD 1024GB,80.0,"1 x Type-C, 2 x USB-A",2.010,129990000.0,...,20.0,i9,12900H,Intel Core i9-12900H,14,20,100.0,100.0,103.0,106.0
559,Razer Blade 17 2022,"Intel Core i9 12900H, 14 nhân, 20 luồng","32GB, 4800 MHz","17.3"", 3840 x 2160 px, IPS, 100% sRGB, 144 Hz","Intel Iris Xe Graphics, Nvidia Geforce RTX3080Ti",SSD 1024GB,82.0,"2 x Type-C, 3 x USB-A, Thunderbolt",2.750,129990000.0,...,20.0,i9,12900H,Intel Core i9-12900H,14,20,100.0,100.0,103.0,106.0


In [513]:
df.rename(columns = {"brand":"cpu_brand"}, inplace = True)

#### Merge the gpu of the laptops with the corresponding score

In [514]:
df["gpu"] = df["gpu"].str.lower()
df["gpu"] = df["gpu"].apply(bs_replace)
df["gpu"] = df["gpu"].str.replace("®", "").str.replace("™", "")
df["gpu"] = df["gpu"].str.split(r" |-")

In [515]:
df["gpu_score"] = np.nan

def gpu_process(df_laptop88):
    for index_gpu, i in enumerate(df_laptop88["gpu"]):
        for index, a_string in enumerate(df_gpu["gpu"]):
            try:
                if all(_ in a_string for _ in i):
                    df_laptop88.loc[index_gpu, "gpu_score"] = df_gpu["gpu_score"].iloc[index]
            except:
                df_laptop88.loc[index_gpu, "gpu_score"] = np.nan
                
gpu_process(df)

In [516]:
df

,name,cpu,ram,monitor,gpu,memory,battery,connection,weight,price,...,level,ver,cpu_name_y,number_of_core,number_of_thread,avg_bench,gamming,desktop,workstation,gpu_score
0,Sạc Macbook Hyperjuice Ultimate Charger 110W DUAL WIRELESS CHARGER (HJ-UC5-W15LR),"Intel Core i5 1235U, 10 nhân, 12 luồng","8GB, DDR4, 3200 MHz","14"", 1920 x 1080 px, IPS, 65% sRGB, 60 Hz","[intel, iris, xe, graphics]",SSD 512GB,39.3,"1 x Type-C, 1 x USB-A",1.400,3590000.0,...,i5,1235U,Intel Core i5-1235U,10,12,63.6,64.0,72.0,59.0,8.34
1,Ghế Công Thái Học HyperWork HW01 (Arm 3D - Gray),"Intel Core i5 11300H, 4 nhân, 8 luồng","16GB, LPDDR4X, 4266 MHz","14.4"", 2400 x 1600 px, PixelSense Flow Display, Cảm ứng, 100% sRGB, 120 Hz","[intel, iris, xe, graphics]",SSD 512GB,58.0,2 x Type-C,1.743,3790000.0,...,i5,11300H,Intel Core i5 11300H,4,8,66.4,66.0,73.0,57.0,8.34
2,Màn hình chuyên đồ hoạ ViewSonic VG2755-2K (QHD/IPS/60Hz),"AMD Ryzen R5-5625U, 6 nhân, 12 luồng","8GB, DDR4, 3200 MHz","15.6"", 1920 x 1080 px, IPS, Chống chói, 65% sRGB","[amd, radeon, graphics]",SSD 512GB,39.3,"1 x Type-C, 1 x USB-A",1.700,7790000.0,...,R5,5625U,AMD Ryzen R5-5625U,6,12,0.0,0.0,0.0,0.0,3.59
3,Asus Vivobook X515 (Chính Hãng) (X515MA-BR481T),"Intel Celeron N4020, 2 nhân, 2 luồng","4GB, DDR4, 3200 MHz","15.6"", 1366 x 768 px, TN, Matte, 60 Hz","[intel, uhd, graphics]","SSD 256GB, HDD 1024GB",37.0,"1 x Type-C, 3 x USB-A",NaN,7990000.0,...,NaN,N4020,Intel Celeron N4020,2,2,20.9,21.0,27.0,15.0,13.90
4,ASUS Vivobook X515JA,"Intel Core i3 1005G1, 2 nhân, 4 luồng","8GB, 3200 MHz","15.6"", 1366 x 768 px, TN, Chống chói, 45% sRGB","[intel, uhd, graphics]",SSD 256GB,37.0,"1 x Type-C, 2 x USB-A",1.800,8990000.0,...,i3,1005G1,Intel Core i3 1005G1,2,4,90.6,91.0,88.0,75.0,13.90
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
556,Razer Blade 15 Advanced (Intel Gen 11) (RZ09-0409CEC3-R3U1),"Intel Core i9 11900H, 8 nhân, 16 luồng","32GB, DDR4, 3200 MHz","15.6"", 3840 x 2160 px, OLED, Glossy, Cảm ứng, 60 Hz","[intel, uhd]",SSD 1024GB,80.0,"2 x Type-C, 3 x USB-A, Thunderbolt",2.010,114990000.0,...,i9,11900H,Intel Core i9 11900H,8,16,90.7,91.0,92.0,91.0,13.90
557,MSI Stealth GS77 2022,"Intel Core i9-12900H, 16 nhân, 24 luồng","32GB, DDR5, 4800 MHz","17.3"", 3840x2160 px, IPS, Chống chói, 120 Hz","[nvidia, geforce, rtx, 3080ti, 16gb, gddr6]",SSD 2048GB,99.9,"2 x Type-C, 2 x USB-A, Thunderbolt",2.800,119990000.0,...,i9,12900H,Intel Core i9-12900H,14,20,100.0,100.0,103.0,106.0,139.00
558,Razer Blade 15 2022,"Intel Core i9 12900H, 14 nhân, 20 luồng","32GB, DDR5, 4800 MHz","15.6"", 3840 x 2160 px, IPS, 100% sRGB, 144 Hz","[intel, iris, xe, graphics, eligible,, rtx30080ti]",SSD 1024GB,80.0,"1 x Type-C, 2 x USB-A",2.010,129990000.0,...,i9,12900H,Intel Core i9-12900H,14,20,100.0,100.0,103.0,106.0,139.00
559,Razer Blade 17 2022,"Intel Core i9 12900H, 14 nhân, 20 luồng","32GB, 4800 MHz","17.3"", 3840 x 2160 px, IPS, 100% sRGB, 144 Hz","[intel, iris, xe, graphics,, nvidia, geforce, rtx3080ti]",SSD 1024GB,82.0,"2 x Type-C, 3 x USB-A, Thunderbolt",2.750,129990000.0,...,i9,12900H,Intel Core i9-12900H,14,20,100.0,100.0,103.0,106.0,139.00


In [517]:
df.isnull().sum()

name                  0
cpu                   0
ram                   1
monitor               1
gpu                   4
memory                1
battery              16
connection            1
weight               56
price                 0
website               0
laptop_brand          0
monitor_size          3
ram_capacity          2
ram_type            133
HDD_capacity          1
SSD_capacity          1
cpu_brand             0
cpu_name_x            0
cpu_core_num          7
cpu_thread_num       17
level                37
ver                   5
cpu_name_y            0
number_of_core        0
number_of_thread      0
avg_bench             0
gamming               0
desktop               0
workstation           0
gpu_score            27
dtype: int64

In [518]:
df_laptop88.columns

Index(['name', 'cpu', 'ram', 'memory', 'gpu', 'battery', 'weight', 'price',
       'website', 'monitor_size', 'ram_capacity', 'ram_type', 'HDD_capacity',
       'SSD_capacity', 'cpu_core_num', 'cpu_thread_num', 'avg_bench',
       'gamming', 'desktop', 'workstation', 'gpu_score', 'laptop_brand',
       'cpu_brand'],
      dtype='object')

In [519]:
df.columns

Index(['name', 'cpu', 'ram', 'monitor', 'gpu', 'memory', 'battery',
       'connection', 'weight', 'price', 'website', 'laptop_brand',
       'monitor_size', 'ram_capacity', 'ram_type', 'HDD_capacity',
       'SSD_capacity', 'cpu_brand', 'cpu_name_x', 'cpu_core_num',
       'cpu_thread_num', 'level', 'ver', 'cpu_name_y', 'number_of_core',
       'number_of_thread', 'avg_bench', 'gamming', 'desktop', 'workstation',
       'gpu_score'],
      dtype='object')

In [520]:
df = df.drop(columns = ["connection","monitor","cpu_name_x","cpu_name_y", "number_of_core", "number_of_thread", "level","ver"])

In [521]:
df.columns

Index(['name', 'cpu', 'ram', 'gpu', 'memory', 'battery', 'weight', 'price',
       'website', 'laptop_brand', 'monitor_size', 'ram_capacity', 'ram_type',
       'HDD_capacity', 'SSD_capacity', 'cpu_brand', 'cpu_core_num',
       'cpu_thread_num', 'avg_bench', 'gamming', 'desktop', 'workstation',
       'gpu_score'],
      dtype='object')

In [522]:
df_laptop88.columns

Index(['name', 'cpu', 'ram', 'memory', 'gpu', 'battery', 'weight', 'price',
       'website', 'monitor_size', 'ram_capacity', 'ram_type', 'HDD_capacity',
       'SSD_capacity', 'cpu_core_num', 'cpu_thread_num', 'avg_bench',
       'gamming', 'desktop', 'workstation', 'gpu_score', 'laptop_brand',
       'cpu_brand'],
      dtype='object')

### Merge the data from both website (thinkpro and laptop88)

In [523]:
concat_df = pd.concat([df,df_laptop88])

In [524]:
concat_df = concat_df.reset_index(drop=True)

In [525]:
concat_df.rename(columns = {"brand": "cpu_brand"}, inplace = True)
concat_df.head()

,name,cpu,ram,gpu,memory,battery,weight,price,website,laptop_brand,...,HDD_capacity,SSD_capacity,cpu_brand,cpu_core_num,cpu_thread_num,avg_bench,gamming,desktop,workstation,gpu_score
0,Sạc Macbook Hyperjuice Ultimate Charger 110W DUAL WIRELESS CHARGER (HJ-UC5-W15LR),"Intel Core i5 1235U, 10 nhân, 12 luồng","8GB, DDR4, 3200 MHz","[intel, iris, xe, graphics]",SSD 512GB,39.3,1.4,3590000.0,thinkpro,Other,...,0,512,Intel,10.0,12.0,63.6,64.0,72.0,59.0,8.34
1,Ghế Công Thái Học HyperWork HW01 (Arm 3D - Gray),"Intel Core i5 11300H, 4 nhân, 8 luồng","16GB, LPDDR4X, 4266 MHz","[intel, iris, xe, graphics]",SSD 512GB,58.0,1.743,3790000.0,thinkpro,Other,...,0,512,Intel,4.0,8.0,66.4,66.0,73.0,57.0,8.34
2,Màn hình chuyên đồ hoạ ViewSonic VG2755-2K (QHD/IPS/60Hz),"AMD Ryzen R5-5625U, 6 nhân, 12 luồng","8GB, DDR4, 3200 MHz","[amd, radeon, graphics]",SSD 512GB,39.3,1.7,7790000.0,thinkpro,Other,...,0,512,AMD,6.0,12.0,0.0,0.0,0.0,0.0,3.59
3,Asus Vivobook X515 (Chính Hãng) (X515MA-BR481T),"Intel Celeron N4020, 2 nhân, 2 luồng","4GB, DDR4, 3200 MHz","[intel, uhd, graphics]","SSD 256GB, HDD 1024GB",37.0,NaN,7990000.0,thinkpro,Asus,...,1024,256,Intel,2.0,2.0,20.9,21.0,27.0,15.0,13.90
4,ASUS Vivobook X515JA,"Intel Core i3 1005G1, 2 nhân, 4 luồng","8GB, 3200 MHz","[intel, uhd, graphics]",SSD 256GB,37.0,1.8,8990000.0,thinkpro,Asus,...,0,256,Intel,2.0,4.0,90.6,91.0,88.0,75.0,13.90


In [526]:
concat_df.loc[concat_df[concat_df["monitor_size"] == "15.6'"].index,"monitor_size"] = "15.6"

In [ ]:
concat_df.to_csv("../data/laptop_concat.csv")

### Change the columns to the right data types

In [527]:
concat_df.dtypes

name               object
cpu                object
ram                object
gpu                object
memory             object
battery            object
weight             object
price              object
website            object
laptop_brand       object
monitor_size       object
ram_capacity       object
ram_type           object
HDD_capacity       object
SSD_capacity       object
cpu_brand          object
cpu_core_num       object
cpu_thread_num     object
avg_bench         float64
gamming           float64
desktop           float64
workstation       float64
gpu_score         float64
dtype: object

In [528]:
concat_df["HDD_capacity"] = concat_df["HDD_capacity"].astype(float)
concat_df["SSD_capacity"] = concat_df["SSD_capacity"].astype(float)
concat_df["monitor_size"] = concat_df["monitor_size"].astype(float)
concat_df["price"] = concat_df["price"].astype(float)
concat_df["weight"] = concat_df["weight"].astype(float)
concat_df["battery"] = concat_df["battery"].astype(float)
concat_df["cpu_core_num"] = concat_df["cpu_core_num"].astype(float)
concat_df["cpu_thread_num"] = concat_df["cpu_thread_num"].astype(float)
concat_df["ram_capacity"] = concat_df["ram_capacity"].astype(float)


In [529]:
concat_df.isnull().sum()

name                0
cpu                 0
ram                 1
gpu                 4
memory              1
battery           163
weight            182
price               0
website             0
laptop_brand        0
monitor_size        3
ram_capacity        2
ram_type          148
HDD_capacity        1
SSD_capacity        1
cpu_brand           0
cpu_core_num      114
cpu_thread_num    124
avg_bench         107
gamming           107
desktop           107
workstation       107
gpu_score          67
dtype: int64

In [530]:
concat_df.drop(["gpu", "memory","ram", "cpu"], axis = 1, inplace = True)
concat_df

,name,battery,weight,price,website,laptop_brand,monitor_size,ram_capacity,ram_type,HDD_capacity,SSD_capacity,cpu_brand,cpu_core_num,cpu_thread_num,avg_bench,gamming,desktop,workstation,gpu_score
0,Sạc Macbook Hyperjuice Ultimate Charger 110W DUAL WIRELESS CHARGER (HJ-UC5-W15LR),39.3,1.400,3590000.0,thinkpro,Other,14.0,8.0,DDR4,0.0,512.0,Intel,10.0,12.0,63.6,64.0,72.0,59.0,8.34
1,Ghế Công Thái Học HyperWork HW01 (Arm 3D - Gray),58.0,1.743,3790000.0,thinkpro,Other,14.4,16.0,NaN,0.0,512.0,Intel,4.0,8.0,66.4,66.0,73.0,57.0,8.34
2,Màn hình chuyên đồ hoạ ViewSonic VG2755-2K (QHD/IPS/60Hz),39.3,1.700,7790000.0,thinkpro,Other,15.6,8.0,DDR4,0.0,512.0,AMD,6.0,12.0,0.0,0.0,0.0,0.0,3.59
3,Asus Vivobook X515 (Chính Hãng) (X515MA-BR481T),37.0,NaN,7990000.0,thinkpro,Asus,15.6,4.0,DDR4,1024.0,256.0,Intel,2.0,2.0,20.9,21.0,27.0,15.0,13.90
4,ASUS Vivobook X515JA,37.0,1.800,8990000.0,thinkpro,Asus,15.6,8.0,NaN,0.0,256.0,Intel,2.0,4.0,90.6,91.0,88.0,75.0,13.90
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
873,[Mới 100% Full Box] Laptop Gaming Dell Alienware M15 R5 Ryzen Edition - R9 5900HX RTX 3070,NaN,2.000,54990000.0,laptop88,Dell,15.6,16.0,DDR4,0.0,1024.0,Intel,NaN,NaN,NaN,NaN,NaN,NaN,112.00
874,[Mới 100% Full Box] Laptop Gigabyte U4 UD 70S1823SO - Intel Core i7,NaN,NaN,25990000.0,laptop88,Other,14.0,16.0,DDR4,0.0,512.0,Intel,4.0,8.0,67.3,67.0,77.0,56.0,8.34
875,[Mới 100% Full Box] Laptop GIGABYTE AORUS 15P YD-73S1224GH - Intel Core i7,NaN,2.000,58990000.0,laptop88,GIGABYTE,15.6,16.0,DDR4,0.0,1024.0,Intel,8.0,16.0,87.9,88.0,90.0,86.0,128.00
876,[Mới 100% Full Box] Laptop Alienware m15 R6 P109F001BBL - Intel Core i7,86.0,2.000,58990000.0,laptop88,Dell,15.6,32.0,DDR4,0.0,1024.0,Intel,8.0,16.0,87.9,88.0,90.0,86.0,NaN


In [531]:
concat_df.dtypes

name               object
battery           float64
weight            float64
price             float64
website            object
laptop_brand       object
monitor_size      float64
ram_capacity      float64
ram_type           object
HDD_capacity      float64
SSD_capacity      float64
cpu_brand          object
cpu_core_num      float64
cpu_thread_num    float64
avg_bench         float64
gamming           float64
desktop           float64
workstation       float64
gpu_score         float64
dtype: object

In [532]:
concat_df.isnull().sum()

name                0
battery           163
weight            182
price               0
website             0
laptop_brand        0
monitor_size        3
ram_capacity        2
ram_type          148
HDD_capacity        1
SSD_capacity        1
cpu_brand           0
cpu_core_num      114
cpu_thread_num    124
avg_bench         107
gamming           107
desktop           107
workstation       107
gpu_score          67
dtype: int64

In [533]:
concat_df.columns

Index(['name', 'battery', 'weight', 'price', 'website', 'laptop_brand',
       'monitor_size', 'ram_capacity', 'ram_type', 'HDD_capacity',
       'SSD_capacity', 'cpu_brand', 'cpu_core_num', 'cpu_thread_num',
       'avg_bench', 'gamming', 'desktop', 'workstation', 'gpu_score'],
      dtype='object')

In [534]:
cols = ['name','website', 'battery', 'weight', 'laptop_brand', 'monitor_size',
       'ram_capacity', 'ram_type', 'HDD_capacity', 'SSD_capacity', 'cpu_brand',
       'cpu_core_num', 'cpu_thread_num', 'avg_bench', 'gamming', 'desktop',
       'workstation', 'gpu_score', 'price']

In [535]:
concat_df = concat_df[cols]

In [536]:
concat_df.shape

(878, 19)

In [537]:
concat_df.head()

,name,website,battery,weight,laptop_brand,monitor_size,ram_capacity,ram_type,HDD_capacity,SSD_capacity,cpu_brand,cpu_core_num,cpu_thread_num,avg_bench,gamming,desktop,workstation,gpu_score,price
0,Sạc Macbook Hyperjuice Ultimate Charger 110W DUAL WIRELESS CHARGER (HJ-UC5-W15LR),thinkpro,39.3,1.400,Other,14.0,8.0,DDR4,0.0,512.0,Intel,10.0,12.0,63.6,64.0,72.0,59.0,8.34,3590000.0
1,Ghế Công Thái Học HyperWork HW01 (Arm 3D - Gray),thinkpro,58.0,1.743,Other,14.4,16.0,NaN,0.0,512.0,Intel,4.0,8.0,66.4,66.0,73.0,57.0,8.34,3790000.0
2,Màn hình chuyên đồ hoạ ViewSonic VG2755-2K (QHD/IPS/60Hz),thinkpro,39.3,1.700,Other,15.6,8.0,DDR4,0.0,512.0,AMD,6.0,12.0,0.0,0.0,0.0,0.0,3.59,7790000.0
3,Asus Vivobook X515 (Chính Hãng) (X515MA-BR481T),thinkpro,37.0,NaN,Asus,15.6,4.0,DDR4,1024.0,256.0,Intel,2.0,2.0,20.9,21.0,27.0,15.0,13.90,7990000.0
4,ASUS Vivobook X515JA,thinkpro,37.0,1.800,Asus,15.6,8.0,NaN,0.0,256.0,Intel,2.0,4.0,90.6,91.0,88.0,75.0,13.90,8990000.0


In [538]:
concat_df.to_csv("../data/laptop_concat.csv", index=False, encoding='utf-8-sig')